In [4]:
import tkinter as tk
import time as time
import math
import random

class ConvexHullApp:
    def __init__(self, master, width, height):
        self.master = master
        self.master.title("Convex Hull Algorithms")
        self.canvas = tk.Canvas(master, width=width, height=height, bg="white")
        self.canvas.pack()

        self.points = []
        self.convex_hull = []
        self.start_time = time.time()

        self.canvas.bind("<Button-1>", self.add_point)

        # Algorithm selection dropdown menu
        self.algorithm_var = tk.StringVar()
        self.algorithm_var.set("Graham Scan")  # Default selection
        algorithms = ["Graham Scan", "Jarvis March"]
        self.algorithm_menu = tk.OptionMenu(master, self.algorithm_var, *algorithms)
        self.algorithm_menu.pack()

        self.draw_button = tk.Button(master, text="Draw Convex Hull", command=self.draw_convex_hull)
        self.draw_button.pack()

        self.time_label = tk.Label(master, text="Time Elapsed:  seconds")
        self.time_label.pack()

        self.reset_button = tk.Button(master, text="Reset", command=self.reset_canvas)
        self.reset_button.pack()

    def update_time_label(self):
        elapsed_time = time.time() - self.start_time
        self.time_label.config(text=f"Time Elapsed: {round(elapsed_time, 10)} seconds")
    
    def draw_line_segment(self, p1, p2, color="red"):
        x1, y1 = p1
        x2, y2 = p2
        self.canvas.create_line(x1, y1, x2, y2, fill=color, width=2, tags="convex_hull")

    def add_point(self, event):
        x, y = event.x, event.y
        self.points.append((x, y))
        self.canvas.create_oval(x - 3, y - 3, x + 3, y + 3, fill="white")
        self.canvas.create_text(x + 10, y - 10, text=f"({x}, {y})", anchor="w", fill="blue")

    def draw_convex_hull(self):
        if len(self.points) < 3:
            return

        # Delete the previous convex hull
        self.canvas.delete("convex_hull")

        algorithm = self.algorithm_var.get()

        if algorithm == "Graham Scan":
            self.convex_hull = self.graham_scan()
        elif algorithm == "Jarvis March":
            self.convex_hull = self.jarvis_march_convex_hull()
        
        # Draw the convex hull
        if self.convex_hull != None:
            for frame in range(1, len(self.convex_hull)+1):
                if frame == len(self.convex_hull):
                    frame = 0
                def update(frame):
                    c0 = self.convex_hull[frame - 1]
                    c1 = self.convex_hull[frame]
                    self.canvas.create_line(c0[0],c0[1], c1[0], c1[1], fill='red', width=2)
                self.canvas.after(500 * frame, update, frame)
        
        
    def reset_canvas(self):
        # Reset canvas by deleting all points and the convex hull
        self.canvas.delete("all")
        self.points = []
        self.convex_hull = []
        self.start_time = time.time()
        self.update_time_label()
        
    
    def graham_scan(self):
        start=time.time()
        # Implement the graham_scan function as before
        global anchor # to be set, (x,y) with smallest y value

        def polar_angle(p0,p1=None):
            if p1==None: p1=anchor
            y_span=p0[1]-p1[1]
            x_span=p0[0]-p1[0]
            return math.atan2(y_span,x_span)

        def distance(p0,p1=None):
            if p1==None: p1=anchor
            y_span=p0[1]-p1[1]
            x_span=p0[0]-p1[0]
            return y_span**2 + x_span**2
        
        def quicksort(a):
            if len(a)<=1:
                return a
            smaller= []
            equal=[]
            larger =[]
            piv_ang=polar_angle(a[random.randint(0,len(a)-1)]) # select random pivot
            for pt in a:
                pt_ang=polar_angle(pt) # calculate current point angle
                if   pt_ang<piv_ang:  smaller.append(pt)
                elif pt_ang==piv_ang: equal.append(pt)
                else:larger.append(pt)
            return  quicksort(smaller) \
                +sorted(equal,key=distance) \
                    +quicksort(larger)

	    # Find the (x,y) point with the lowest y value,
	    # along with its index in the 'points' list. If
	    # there are multiple points with the same y value,
	    # choose the one with smallest x.
        min_idx=None
        for i,(x,y) in enumerate(self.points):
            if min_idx==None or y<self.points[min_idx][1]:
                min_idx=i
            if y==self.points[min_idx][1] and x<self.points[min_idx][0]:
                min_idx=i

	    # set the global variable 'anchor', used by the
	    # 'polar_angle' and 'distance' functions
        anchor= self.points[min_idx]

	    # sort the points by polar angle then delete
	    # the anchor from the sorted list
        sorted_pts=quicksort(self.points)
        del sorted_pts[sorted_pts.index(anchor)]

        def det(p1,p2,p3):
            return   (p2[0]-p1[0])*(p3[1]-p1[1]) \
			-(p2[1]-p1[1])*(p3[0]-p1[0])

	    # anchor and point with smallest polar angle will always be on hull
        hull=[anchor,sorted_pts[0]]
        for s in sorted_pts[1:]:
            while det(hull[-2],hull[-1],s)<=0:
                del hull[-1] # backtrack
			#if len(hull)<2: break
            hull.append(s)
        self.update_time_label()
        return hull


    def jarvis_march_convex_hull(self):
        start = time.time()
        def orientation(p, q, r):
            val = (q[1] - p[1]) * (r[0] - q[0]) - (q[0] - p[0]) * (r[1] - q[1])
            if val == 0:
                return 0
            return 1 if val > 0 else -1
    # Implementation of jarvis_march_convex_hull function as before
        n = len(self.points)
        if n < 3:
            raise ValueError("Convex hull not possible with less than 3 points")

    # Find the point with the lowest y-coordinate (and leftmost if ties)
        pivot = min(self.points, key=lambda p: (p[1], p[0]))

        hull = []  # Convex hull to be returned

        while True:
            hull.append(pivot)
            endpoint = self.points[0]

            for i in range(1, n):
                if endpoint == pivot or orientation(pivot, endpoint, self.points[i]) == -1:
                    endpoint = self.points[i]

            if endpoint == hull[0]:
                break  # Convex hull is complete

            pivot = endpoint
        self.update_time_label()
        return hull

if __name__ == "__main__":
    root = tk.Tk()
    app = ConvexHullApp(root, width=900, height=500)
    root.mainloop()
